# XAI Pipeline for Skin Cancer Classification
This notebook loads the trained Xception model, applies there XAI methods (Grad-CAM, SHAP, Influence Functions), and exports the results in a format ready for the LLM integration pipeline.

## Import Libraries

In [ ]:
# --- 1. Setup and Initialization ---
import sys
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm

from torch.utils.data import DataLoader, TensorDataset
from torch import nn
from torch.nn import functional as F
from captum.attr import LayerGradCam, Occlusion
from captum.attr import visualization as viz

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
if project_root not in sys.path:
    sys.path.append(project_root)
    print(f"Project root added to path: {project_root}")

# Import the new helper functions
from scd.utils.common import load_model, load_datasets, get_test_transforms
print("Libraries imported successfully.")

## Define Project Structure and File Paths

In [ ]:
# --- 2. Define Project Structure and File Paths ---
BASE_DIR = project_root
MODEL_PATH = os.path.join(BASE_DIR, 'models', 'ResNet_skin_cancer_classification.pth')
PROCESSED_DATA_DIR = os.path.join(BASE_DIR, 'data', 'processed')
SAMPLE_IMAGE_PATH = os.path.join(BASE_DIR, 'user_inputs', 'user_sample1.jpg') # Assuming a sample image exists here
OUTPUT_DIR = os.path.join(BASE_DIR, 'results', 'xai_output')
os.makedirs(OUTPUT_DIR, exist_ok=True)

# New constant for image size
IMAGE_RESIZE = (384, 384)

print(f"Project Base Directory: {BASE_DIR}")
print(f"Using Model Path: {MODEL_PATH}")

## Model Definition and Loading

In [ ]:
# --- 3. Model Definition and Loading ---
print("\nLoading model...")
# Use the load_model helper function
model = load_model(MODEL_PATH)
model.eval()
print(f"Successfully loaded trained ResNet model.")

## Data Loading and Preprocessing

In [ ]:
# --- 4. Data Loading and Preprocessing ---
print("\nLoading and preprocessing sample image...")
# Use the get_test_transforms helper function
preprocess_transform = get_test_transforms(resize=IMAGE_RESIZE)

try:
    input_image = Image.open(SAMPLE_IMAGE_PATH).convert('RGB')
    input_image = input_image.resize(IMAGE_RESIZE, resample=Image.BILINEAR)
    image_np = np.array(input_image)
    input_tensor = preprocess_transform(image=image_np)['image'].unsqueeze(0)
    print(f"Successfully loaded and preprocessed sample image: {SAMPLE_IMAGE_PATH}")
except FileNotFoundError:
    print(f"ERROR: Sample image not found at '{SAMPLE_IMAGE_PATH}'")

## XAI Pipeline Implementation

In [ ]:
# --- 5. XAI Pipeline Implementation ---
print("\n--- Running XAI Pipeline ---")

# Get model prediction
outputs, attn_map = model(input_tensor)
prediction_score, pred_label_idx = torch.topk(outputs, 1)
predicted_class_index = pred_label_idx.squeeze().item()
class_names = ['Benign', 'Malignant']
print(f"Model Prediction: Class '{class_names[predicted_class_index]}' with confidence {prediction_score.item():.2%}")

# Create and save the model output DataFrame
probabilities = torch.softmax(outputs, dim=1) # apply softmax
benign_prob = probabilities[0][0].item()
malignant_prob = probabilities[0][1].item()

# Create a list of dictionaries for the two rows
output_data = [
    {'class': 'Benign', 'confidence': benign_prob},
    {'class': 'Malignant', 'confidence': malignant_prob}
]
output_df = pd.DataFrame(output_data)

# Save the file
model_output_path = os.path.join(OUTPUT_DIR, 'model_output.csv')
output_df.to_csv(model_output_path, index=False)
print(f"Model output saved to: {model_output_path}")

# Wrapper function for Captum
def model_forward_wrapper(input_tensor):
    """Wrapper to return only the logits tensor from the model's output."""
    return model(input_tensor)[0] # Return only the first element (the predictions)

In [ ]:
# --- 5a. Built-in Attention Map Visualization ---
print("\n--- Visualizing Model's Built-in Attention Map ---")

# Upsample attention map to input image size
upsampled_attn = F.interpolate(attn_map, size=IMAGE_RESIZE, mode='bilinear', align_corners=False)

# Convert to numpy and normalize to [0, 1] range
attn_np = upsampled_attn.squeeze().cpu().detach().numpy()
attn_np = (attn_np - attn_np.min()) / (attn_np.max() - attn_np.min())

# Create the figure and axes
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot original image
axes[0].imshow(input_image)
axes[0].set_title("Original Image")
axes[0].axis("off")

# Plot attention heatmap
im = axes[1].imshow(attn_np, cmap='jet')
axes[1].set_title("Attention Map")
axes[1].axis("off")
cbar = fig.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
cbar.set_label('Attention Intensity', rotation=270, labelpad=15)

# Create overlay of attention on original image
# Convert input image to numpy for overlay
input_np = np.array(input_image)
# Create a colored heatmap using matplotlib's colormap
cmap = plt.cm.jet
heatmap_colored = cmap(attn_np)[:, :, :3]  # Get RGB from colormap (drop alpha)
# Create overlay (blend original image with heatmap)
overlay = 0.7 * input_np / 255.0 + 0.3 * heatmap_colored

# Plot overlay
axes[2].imshow(overlay)
axes[2].set_title("Overlay (30% attention, 70% image)")
axes[2].axis("off")

plt.tight_layout()

# Save the visualization
grad_cam_output_path = os.path.join(OUTPUT_DIR, 'sample3_attn.png')
plt.savefig(grad_cam_output_path, bbox_inches='tight', pad_inches=0)
plt.show()

### Grad-CAM Implementation

In [ ]:
# --- 5a. Grad-CAM Implementation ---
print("\n--- Generating Grad-CAM ---")
# Target layer for ResNet model from scd/model.py
grad_cam_layer = model.features[-1] 
layer_gc = LayerGradCam(model_forward_wrapper, grad_cam_layer) # Use the wrapper
attribution_gc = layer_gc.attribute(input_tensor, target=predicted_class_index)

# Upsample the heatmap to the original image size
heatmap = F.interpolate(attribution_gc, size=input_image.size, mode='bilinear', align_corners=False)
heatmap = heatmap.squeeze().cpu().detach().numpy()

# Plotting
fig, ax = plt.subplots(figsize=(8, 8))
ax.set_title('Grad-CAM Visualisation')
ax.imshow(input_image)
# Capture the image object returned by imshow to use in the color bar
im = ax.imshow(heatmap, cmap='jet', alpha=0.5) 
ax.axis('off')

# Add the color bar to the figure
cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label('Importance for Predicted Class', rotation=270, labelpad=20)

# Save and show the plot
grad_cam_output_path = os.path.join(OUTPUT_DIR, 'user_sample1_xai_gradcam.png')
plt.savefig(grad_cam_output_path, bbox_inches='tight', pad_inches=0)
plt.show()
print(f"Grad-CAM image saved to: {grad_cam_output_path}")

### SHAP

In [ ]:
# --- 5b. SHAP (using Occlusion) ---
print("\n--- Generating SHAP ---")
occlusion = Occlusion(model_forward_wrapper) # Use the wrapper here
# Adjust sliding window shapes for the new 384x384 size
attribution_shap = occlusion.attribute(input_tensor, 
                                       strides=(3, 32, 32), 
                                       target=predicted_class_index, 
                                       sliding_window_shapes=(3, 48, 48),
                                       baselines=0)

# The visualize_image_attr function creates the plot and color bar
fig, _ = viz.visualize_image_attr(
    attr=np.transpose(attribution_shap.squeeze().cpu().detach().numpy(), (1,2,0)), 
    original_image=np.array(input_image.resize(IMAGE_RESIZE)), 
    method="blended_heat_map", 
    sign="all", 
    show_colorbar=True,
    title="SHAP Explanation"
)

# Save and show the plot
shap_output_path = os.path.join(OUTPUT_DIR, 'user_sample1_xai_shap.png')
fig.savefig(shap_output_path, bbox_inches='tight', pad_inches=0)
plt.show()
print(f"SHAP image saved to: {shap_output_path}")

### Influence Functions

In [ ]:
# --- 5c. Influence Functions ---
print("\n--- Generating Influence Functions ---")
def get_gradient_and_prediction(model, data_tensor, target, loss_fn):
    model.zero_grad()
    # Use the wrapper to get logits
    logits = model_forward_wrapper(data_tensor)
    _, pred_idx = torch.max(logits.data, 1)
    loss = loss_fn(logits, target)
    loss.backward()
    # Get gradient from the final classifier layer of the ResNet model
    grad = model.classifier[-1].weight.grad.detach().clone()
    return grad.flatten(), pred_idx.item()

def calculate_real_influence(model, train_loader, test_tensor, test_target, device):
    model.to(device)
    model.eval()
    test_tensor = test_tensor.to(device)
    test_target = torch.tensor([test_target]).to(device)
    loss_fn = nn.CrossEntropyLoss()
    
    print("Calculating gradient for the test image...")
    test_grad, _ = get_gradient_and_prediction(model, test_tensor, test_target, loss_fn)
    
    results = []
    print("Iterating through the training dataset to calculate influence scores...")
    for (train_imgs, train_labels, filenames) in tqdm(train_loader):
        train_imgs, train_labels = train_imgs.to(device), train_labels.to(device)
        for i in range(len(train_imgs)):
            train_grad, train_pred = get_gradient_and_prediction(model, train_imgs[i].unsqueeze(0), train_labels[i].unsqueeze(0), loss_fn)
            influence_score = torch.dot(test_grad, train_grad).item()
            results.append({'score': influence_score, 'prediction': train_pred, 'filename': filenames[i]})
    return results

try:
    print(f"Loading processed training data from: {PROCESSED_DATA_DIR}")
    # Load data using the new helper function
    train_dataset, train_filenames = load_datasets(PROCESSED_DATA_DIR, only_train_dataset_with_filenames=True)
    
    # Create a new TensorDataset that includes filenames for the loader
    class DatasetWithFilenames(TensorDataset):
        def __init__(self, tensors, filenames):
            super(DatasetWithFilenames, self).__init__(*tensors)
            self.filenames = filenames
        def __getitem__(self, index):
            return super(DatasetWithFilenames, self).__getitem__(index) + (self.filenames[index],)

    train_images_tensors, train_labels_tensors = train_dataset.tensors
    train_dataset_with_files = DatasetWithFilenames((train_images_tensors, train_labels_tensors), train_filenames)
    train_loader = DataLoader(train_dataset_with_files, batch_size=32)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    influence_results = calculate_real_influence(model, train_loader, input_tensor, predicted_class_index, device)
    
    # The ground truth is now part of the dataset, not a separate CSV
    gt_map = {fname: label.item() for fname, label in zip(train_filenames, train_labels_tensors)}

    report_data = [{'case_id': r['filename'].split('.')[0], 
                    'influence_score': r['score'], 
                    'ground_truth': 'Malignant' if gt_map.get(r['filename'], -1) == 1 else 'Benign', # Look up ground truth
                    'prediction': 'Malignant' if r['prediction'] == 1 else 'Benign'} for r in influence_results]
    
    report_df = pd.DataFrame(report_data)
    report_df['abs_influence'] = report_df['influence_score'].abs()
    report_df = report_df.sort_values(by='abs_influence', ascending=False).drop(columns='abs_influence')
    final_report_df = report_df.head(100)

    influence_output_path = os.path.join(OUTPUT_DIR, 'user_sample1_influence_function.csv')
    final_report_df.to_csv(influence_output_path, index=False)
    print(f"\nSuccessfully saved {len(final_report_df)} influence scores to: {influence_output_path}")

except FileNotFoundError as e:
    print(f"ERROR: A required data file was not found. Please ensure notebooks 01 and 02 have been run. Details: {e}")

print("\n\n--- XAI Pipeline Execution Complete ---")